In [ ]:
!pip install pyspark


     |████████████████████████████████| 281.3 MB 37 kB/s 
     |████████████████████████████████| 198 kB 14.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=ff249232e6fb5c98a090799a97c6874757c7458e210871b86053e324e7de1f45
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [ ]:
import pandas as pd
import numpy as np

#set print width
pd.set_option('display.max_colwidth', 200)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#import dataset
from ast import literal_eval
df = pd.read_csv('/content/drive/MyDrive/SOdata/SOdata-Tokenized.csv',
                 converters={'tags': literal_eval, 'tokenized_text': literal_eval, 'tokenized_title': literal_eval})
df.head()

FileNotFoundError: ignored

In [ ]:
for index in df.index :
  text_list = df['tokenized_text'][index]
  text_str = " ".join(text_list).strip()
  df['tokenized_text'][index] = text_str
  title_list = df['tokenized_title'][index]
  title_str = " ".join(title_list).strip()
  df['tokenized_title'][index] = title_str
  print(index)

In [ ]:
df.head()

In [ ]:
df.to_csv('data.csv', index=False)
!cp data.csv "/content/drive/MyDrive/SOdata/data.csv"

Create the labels

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['tags'])
y.shape

In [ ]:
len(y[2])

Creating Train and Test set : *Two pairs of sets for the text/body and the title*

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['tokenized_title'], y, random_state=42, test_size=0.30, shuffle=True)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
'''
# Create Train and test set from the text
train_text = train_set['tokenized_text']
test_text = test_set['tokenized_text']

# Create Train and Test set from the title
train_title = train_set['tokenized_title']
test_title = test_set['tokenized_title']
'''

TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
train_word_features = vectorizer.fit_transform(X_train)
test_word_features = vectorizer.transform(X_test)

Naive Bayes

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score

In [ ]:
y_train.shape

In [ ]:
nb_model=OneVsRestClassifier(MultinomialNB())

nb_model.fit(train_word_features,y_train)

In [ ]:
train_pred_nb=nb_model.predict_proba(train_word_features)

In [ ]:
def classify(pred_prob,threshold):
  y_pred_seq = []

  for i in pred_prob:
    temp=[]
    for j in i:
      if j>=threshold:
        temp.append(1)
      else:
        temp.append(0)
    y_pred_seq.append(temp)

  return y_pred_seq

def optimum_threshold(actual,pred_prob):
  #define candidate threshold values
  thresholds  = np.arange(0,0.5,0.01)

  score=[]
  for value in thresholds:
    # Getting classes for each threshold
    pred_classes= classify(pred_prob,value)
    # Getting F1-score for every threshold
    score.append(f1_score(actual,pred_classes,average="weighted"))

  return thresholds[score.index(max(score))]

print("Optimal threshold=> 0.14",optimum_threshold(y_train,train_pred_nb))
threshold = optimum_threshold(y_train,train_pred_nb)
train_pred_nb_class = classify(train_pred_nb,threshold)
mlb.inverse_transform(np.array(train_pred_nb_class[:5]))

In [ ]:
train_pred_nb_class = classify(train_pred_nb,0.14)
mlb.inverse_transform(np.array(train_pred_nb_class[:5]))

In [ ]:
print("F1-score on Train Set:",f1_score(y_train,train_pred_nb_class,average="weighted"))